# Analisando os dados para ganhar _insights_
<br>
Aqui vamos analisar os dados somente do nosso conjunto de treino para ganhar alguns insights sobre o comportamento das fraudes.

## Configurações Iniciais

In [0]:
# chamando conjunto de treino definido na etapa anterior
train_set = spark.read.table("my_catalog.default.creditcard_treino_tcc")

In [0]:
from pyspark.sql import functions as F

stats_df = (
    train_set.groupBy("Class")
    .agg(
        F.round(F.expr("percentile_approx(Amount, 0.5)"), 2).alias("Mediana"),
        F.round(F.avg("Amount"), 2).alias("Média"),
        F.round(F.stddev("Amount"), 2).alias("Desvio Padrão"),
        F.round(F.stddev("Amount") / F.avg("Amount"), 2).alias("Coef. Variação")
    )
)

display(stats_df)

 | Class | Mediana | Média   | Desvio Padrão | Coef. Variação |
 |-------|---------|---------|---------------|----------------|
 | 0     | 22      | 87.96   | 248.03        | 2.82           |
 | 1     | 9.82    | 128.81  | 267.83        | 2.08           |

In [0]:
#test_counts = test_set.groupBy("Class").count().toPandas()
#train_counts = train_set.groupBy("Class").count().toPandas()
#test_count = test_set.count()
#test_count = test_set.count()
#train_count = train_set.count()

In [0]:
df = train_set.toPandas()

In [0]:
df.head(5)

In [0]:
df.describe()

In [0]:
print(f"Número de linhas: {df.shape[0]}")
print(f"Número de colunas: {df.shape[1]}")

In [0]:
fraud_count = df[df['Class'] == 1].shape[0]
legit_count = df[df['Class'] == 0].shape[0]
print(f"Número de transações fraudulentas: {fraud_count}")
print(f"Número de transações legítimas: {legit_count}")

## Análise Inicial das Variáveis Principais

In [0]:
print(f"Total de transações: {len(df)}")
print(f"Transações normais: {len(df[df['Class'] == 0])}")
print(f"Transações fraudulentas: {len(df[df['Class'] == 1])}")
print(f"Taxa de fraude: {len(df[df['Class'] == 1]) / len(df) * 100:.4f}%")

In [0]:
print(f"\nAmount - Média: {df['Amount'].mean():.2f}")
print(f"Amount - Mediana: {df['Amount'].median():.2f}")
print(f"Amount - Desvio Padrão: {df['Amount'].std():.2f}")
print(f"Amount - Máximo: {df['Amount'].max():.2f}")

In [0]:
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.titleweight'] = 'bold'

In [0]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0,0].hist(df['Amount'], bins=50, alpha=0.7, color='skyblue')
axes[0,0].set_title('\nDistribuição de Amount\n', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Amount')
axes[0,0].set_ylabel('Frequência')

axes[0,1].hist(df['Time'], bins=50, alpha=0.7, color='lightcoral')
axes[0,1].set_title('\nDistribuição de Time\n', fontsize=14, fontweight='bold')
axes[0,1].set_xlabel('Time (segundos)')
axes[0,1].set_ylabel('Frequência')

class_counts = df['Class'].value_counts()
axes[1,0].bar(class_counts.index, class_counts.values, color=['lightblue', 'red'], alpha=0.7)
axes[1,0].set_title('\nDistribuição das Classes\n', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Classe (0: Legítima, 1: Fraude)')
axes[1,0].set_ylabel('Quantidade')
axes[1,0].set_xticks([0, 1])

fraud_amount = df[df['Class'] == 1]['Amount']
normal_amount = df[df['Class'] == 0]['Amount']
axes[1,1].boxplot([normal_amount, fraud_amount], labels=['Legítima', 'Fraude'])
axes[1,1].set_title('\nDistribuição de Amount por Classe\n', fontsize=14, fontweight='bold')
axes[1,1].set_ylabel('Amount')

plt.tight_layout()
plt.show()

In [0]:
import numpy as np

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

axes[0,0].hist(normal_amount, bins=50, alpha=0.5, label='Normal', color='blue')
axes[0,0].hist(fraud_amount, bins=50, alpha=0.5, label='Fraude', color='red')
axes[0,0].set_title('\nDistribuição de Amount - Legítima vs Fraude\n', fontsize=14, fontweight='bold')
axes[0,0].set_xlabel('Amount')
axes[0,0].set_ylabel('Frequência')
axes[0,0].legend()
axes[0,0].set_yscale('log')

amount_data = [normal_amount, fraud_amount]
axes[0,1].boxplot(amount_data, labels=['Legítima', 'Fraude'])
axes[0,1].set_title('\nBoxplot de Amount por Classe\n', fontsize=14, fontweight='bold')
axes[0,1].set_ylabel('Amount')

percentis = np.arange(0, 101, 10)
fraud_percentis = np.percentile(fraud_amount, percentis)
axes[1,0].plot(percentis, fraud_percentis, marker='o', color='red', linewidth=2)
axes[1,0].set_title('\nPercentis de Amount - Transações Fraudulentas\n', fontsize=14, fontweight='bold')
axes[1,0].set_xlabel('Percentil')
axes[1,0].set_ylabel('Amount')
axes[1,0].grid(True, alpha=0.3)

fraud_amount_sorted = np.sort(fraud_amount)
fraud_cdf = np.arange(1, len(fraud_amount_sorted)+1) / len(fraud_amount_sorted)
axes[1,1].plot(fraud_amount_sorted, fraud_cdf, color='red', linewidth=2)
axes[1,1].set_title('\nDistribuição Cumulativa - Amount Fraudulento\n', fontsize=14, fontweight='bold')
axes[1,1].set_xlabel('Amount')
axes[1,1].set_ylabel('Probabilidade Cumulativa')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [0]:
print(f"Amount médio nas fraudes: {fraud_amount.mean():.2f}")
print(f"Amount mediano nas fraudes: {fraud_amount.median():.2f}")
print(f"Amount máximo nas fraudes: {fraud_amount.max():.2f}")
print(f"Amount mínimo nas fraudes: {fraud_amount.min():.2f}")
print(f"Desvio padrão do Amount nas fraudes: {fraud_amount.std():.2f}")
print(f"Coeficiente de variação: {(fraud_amount.std() / fraud_amount.mean() * 100):.2f}%")

In [0]:
# convertendo Time para horas para melhor interpretação
df['Time_hours'] = df['Time'] / 3600

In [0]:
df.head(5)

In [0]:
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# distribuição temporal geral
axes[0,0].hist(df['Time_hours'], bins=48, alpha=0.7, color='lightgreen')
axes[0,0].set_title('\nDistribuição Temporal Geral (Horas)\n')
axes[0,0].set_xlabel('Tempo (horas)')
axes[0,0].set_ylabel('Frequência')

# distribuição temporal das fraudes
fraud_time = df[df['Class'] == 1]['Time_hours']
axes[0,1].hist(fraud_time, bins=48, alpha=0.7, color='red')
axes[0,1].set_title('\nDistribuição Temporal das Fraudes (Horas)\n')
axes[0,1].set_xlabel('Tempo (horas)')
axes[0,1].set_ylabel('Frequência')

# padrão diário (mod 24 horas)
df['Hour_of_day'] = df['Time_hours'] % 24
normal_hours = df[df['Class'] == 0]['Hour_of_day']
axes[1,0].hist(normal_hours, bins=24, alpha=0.8, color='blue')
axes[1,0].set_title('\nDistribuição por Hora do Dia - Transações Legítimas\n')
axes[1,0].set_xlabel('Hora do Dia')
axes[1,0].set_ylabel('Frequência')
axes[1,0].grid(True, alpha=0.3)

fraud_hours = df[df['Class'] == 1]['Hour_of_day']
axes[1,1].hist(fraud_hours, bins=24, alpha=0.8, color='red')
axes[1,1].set_title('\nDistribuição por Hora do Dia - Transações Fraudulentas\n')
axes[1,1].set_xlabel('Hora do Dia')
axes[1,1].set_ylabel('Frequência')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [0]:
# períodos do dia
def periodo_do_dia(hora):
    if 0 <= hora < 6:
        return 'Madrugada'
    elif 6 <= hora < 12:
        return 'Manhã'
    elif 12 <= hora < 18:
        return 'Tarde'
    else:
        return 'Noite'

df['Periodo'] = df['Hour_of_day'].apply(periodo_do_dia)

fig, axes = plt.subplots(1, 2, figsize=(15, 6))

legit_counts = df[df['Class'] == 0]['Periodo'].value_counts().reindex(['Madrugada', 'Manhã', 'Tarde', 'Noite'])
legit_percent = (legit_counts / legit_counts.sum() * 100).round(2)
axes[0].bar(legit_counts.index, legit_counts.values, color='lightblue', alpha=0.8)
axes[0].set_title('\nTransações Legítimas por Período do Dia\n', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Período do Dia')
axes[0].set_ylabel('Quantidade')
axes[0].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(legit_counts.values):
    percent = legit_percent.values[i]
    axes[0].text(i, v + max(legit_counts.values)*0.01, f'{v:,}', ha='center', va='bottom', fontweight='bold', fontsize=8)

fraud_counts = df[df['Class'] == 1]['Periodo'].value_counts().reindex(['Madrugada', 'Manhã', 'Tarde', 'Noite'])
fraud_percent = (fraud_counts / fraud_counts.sum() * 100).round(2)
axes[1].bar(fraud_counts.index, fraud_counts.values, color='red', alpha=0.8)
axes[1].set_title('\nTransações Fraudulentas por Período do Dia\n', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Período do Dia')
axes[1].set_ylabel('Quantidade')
axes[1].grid(True, alpha=0.3, axis='y')
for i, v in enumerate(fraud_counts.values):
    percent = fraud_percent.values[i]
    axes[1].text(i, v + max(fraud_counts.values)*0.01, f'{v:,}', ha='center', va='bottom', fontweight='bold', fontsize=8)

plt.tight_layout()
plt.show()

In [0]:
print("\nANÁLISE POR PERÍODO DO DIA:")
periodo_stats = df.groupby('Periodo').agg({
    'Class': ['count', 'mean']
}).round(4)
periodo_stats.columns = ['Total_Transacoes', 'Taxa_Fraude']
periodo_stats['Taxa_Fraude_Perc'] = (periodo_stats['Taxa_Fraude'] * 100).round(4)
periodo_stats = periodo_stats.reindex(['Madrugada', 'Manhã', 'Tarde', 'Noite'])

print(periodo_stats)

# taxa de fraude por período
print(f"\nPERÍODO COM MAIOR TAXA DE FRAUDE: {periodo_stats['Taxa_Fraude_Perc'].idxmax()} ({periodo_stats['Taxa_Fraude_Perc'].max():.4f}%)")
print(f"PERÍODO COM MENOR TAXA DE FRAUDE: {periodo_stats['Taxa_Fraude_Perc'].idxmin()} ({periodo_stats['Taxa_Fraude_Perc'].min():.4f}%)")

# comparação relativa
max_fraud_period = periodo_stats['Taxa_Fraude_Perc'].idxmax()
min_fraud_period = periodo_stats['Taxa_Fraude_Perc'].idxmin()
ratio = periodo_stats.loc[max_fraud_period, 'Taxa_Fraude_Perc'] / periodo_stats.loc[min_fraud_period, 'Taxa_Fraude_Perc']
print(f"RAZÃO ENTRE MAIOR/MENOR TAXA: {ratio:.2f}x")

### Análise das _features_ v1 a v28

In [0]:
pca_features = [f'V{i}' for i in range(1, 29)]

print("ESTATÍSTICAS DAS FEATURES V1 A V28:")
pca_stats = df[pca_features].describe()
print(pca_stats.loc[['mean', 'std', 'min', 'max', '50%']])

print("\nANÁLISE DE DISTRIBUIÇÃO POR CLASSE:")
for feature in pca_features:
    fraud_mean = df[df['Class'] == 1][feature].mean()
    normal_mean = df[df['Class'] == 0][feature].mean()
    diff = fraud_mean - normal_mean
    print(f"{feature}: Fraude={fraud_mean:7.3f}, Normal={normal_mean:7.3f}, Diff={diff:7.3f}")

fig, axes = plt.subplots(7, 4, figsize=(20, 25))
axes = axes.ravel()

for i, feature in enumerate(pca_features):
    
    axes[i].hist(df[df['Class'] == 0][feature], 
                bins=50, alpha=0.5, label='Normal', color='blue', density=True)
    axes[i].hist(df[df['Class'] == 1][feature], 
                bins=50, alpha=0.5, label='Fraude', color='red', density=True)
    axes[i].set_title(f'Distribuição de {feature}', fontsize=11, fontweight='bold')
    axes[i].set_xlabel(feature, fontsize=9)
    axes[i].set_ylabel('Densidade', fontsize=9)
    axes[i].legend(fontsize=8)
    axes[i].grid(True, alpha=0.3)

for i in range(len(pca_features), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

In [0]:
features = [f'V{i}' for i in range(1, 29)]

fig, ax2 = plt.subplots(figsize=(14, 6))
variance_ordered = df[features].var()

# Cálculo da % de variância por grupo
var_total = variance_ordered.sum()
var_v1_v10 = variance_ordered[:10].sum()
var_v11_v20 = variance_ordered[10:20].sum()
var_v21_v28 = variance_ordered[20:28].sum()

pct_v1_v10 = 100 * var_v1_v10 / var_total
pct_v11_v20 = 100 * var_v11_v20 / var_total
pct_v21_v28 = 100 * var_v21_v28 / var_total

variance_ordered.plot(kind='bar', ax=ax2, color='orange')
ax2.set_title('\nVariância por Feature (V1 a V28)\n', fontsize=14, fontweight='bold')
ax2.set_xlabel('Feature', fontsize=12)
ax2.set_ylabel('Variância', fontsize=12)
ax2.grid(axis='y', alpha=0.3)

ax2.axvspan(-0.5, 9.5, alpha=0.2, color='green', label=f'V1-V10 (Principais) [{pct_v1_v10:.1f}%]')
ax2.axvspan(9.5, 19.5, alpha=0.2, color='yellow', label=f'V11-V20 (Intermediários) [{pct_v11_v20:.1f}%]')
ax2.axvspan(19.5, 27.5, alpha=0.2, color='red', label=f'V21-V28 (Finais) [{pct_v21_v28:.1f}%]')
ax2.legend()

plt.tight_layout()
plt.show()

In [0]:
ranges = df[features].max() - df[features].min()

fig, ax = plt.subplots(figsize=(14, 6))
ranges_ordered = ranges[features]
bars = ax.bar(range(len(features)), ranges_ordered, color='teal', alpha=0.7)

# colorir por grupo
for i in range(10):
    bars[i].set_color('#2ecc71')
for i in range(10, 20):
    bars[i].set_color('#f39c12')
for i in range(20, 28):
    bars[i].set_color('#e74c3c')

ax.set_xticks(range(len(features)))
ax.set_xticklabels(features, rotation=45, ha='right')
ax.set_title('\nAmplitude das Features V1 a V28\n', fontsize=14, fontweight='bold')
ax.set_xlabel('Feature', fontsize=12)
ax.set_ylabel('Amplitude', fontsize=12)
ax.grid(axis='y', alpha=0.3)

from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ecc71', label='V1-V10 (Principais)'),
    Patch(facecolor='#f39c12', label='V11-V20 (Intermediários)'),
    Patch(facecolor='#e74c3c', label='V21-V28 (Finais)')
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

In [0]:
pairplot_features = top_features[:6] + ['Class']

print("ANÁLISE DE INTERAÇÕES ENTRE TOP FEATURES:")

sns.set(style='whitegrid', font_scale=1.2)
pairplot = sns.pairplot(
    df[pairplot_features],
    hue='Class',
    palette={0: 'dodgerblue', 1: 'crimson'},
    diag_kind='kde',
    plot_kws={
        'alpha': 0.5,
        's': 18,
        'edgecolor': 'k',
        'linewidth': 0.3
    },
    diag_kws={
        'shade': True
    },
    corner=True
)
pairplot.fig.suptitle(
    'Pairplot - Top Features vs Classe',
    y=1.03,
    fontsize=18,
    fontweight='bold'
)
for ax in pairplot.axes.flatten():
    if ax:
        ax.grid(True, alpha=0.3)
        ax.set_facecolor('#f9f9f9')
pairplot._legend.set_title('Classe')
pairplot._legend.set_bbox_to_anchor((1, 0.5))
plt.tight_layout()
plt.show()

In [0]:
# combinações de features promissoras
df['V12_V14'] = df['V12'] + df['V14']  
df['V3_V10'] = df['V3'] * df['V10']   


fig, axes = plt.subplots(2, 2, figsize=(15, 10))


axes[0,0].hist(df[df['Class'] == 0]['V12_V14'], bins=50, alpha=0.5, label='Normal', color='blue', density=True)
axes[0,0].hist(df[df['Class'] == 1]['V12_V14'], bins=50, alpha=0.5, label='Fraude', color='red', density=True)
axes[0,0].set_title('Combinação V12 + V14', fontweight='bold')
axes[0,0].legend()


axes[0,1].hist(df[df['Class'] == 0]['V3_V10'], bins=50, alpha=0.5, label='Normal', color='blue', density=True)
axes[0,1].hist(df[df['Class'] == 1]['V3_V10'], bins=50, alpha=0.5, label='Fraude', color='red', density=True)
axes[0,1].set_title('Combinação V3 * V10', fontweight='bold')
axes[0,1].legend()


df['V14_V12_ratio'] = df['V14'] / (df['V12'] + 1e-8)  # Evitar divisão por zero
axes[1,0].hist(df[df['Class'] == 0]['V14_V12_ratio'], bins=50, alpha=0.5, label='Normal', color='blue', density=True)
axes[1,0].hist(df[df['Class'] == 1]['V14_V12_ratio'], bins=50, alpha=0.5, label='Fraude', color='red', density=True)
axes[1,0].set_title('Ratio V14/V12', fontweight='bold')
axes[1,0].legend()


top_pca_features = ['V1', 'V2', 'V3', 'V4', 'V7', 'V10', 'V11', 'V12', 'V14', 'V17']
df['pca_magnitude'] = np.sqrt((df[top_pca_features] ** 2).sum(axis=1))
axes[1,1].hist(df[df['Class'] == 0]['pca_magnitude'], bins=50, alpha=0.5, label='Normal', color='blue', density=True)
axes[1,1].hist(df[df['Class'] == 1]['pca_magnitude'], bins=50, alpha=0.5, label='Fraude', color='red', density=True)
axes[1,1].set_title('Magnitude das Top PCA Features', fontweight='bold')
axes[1,1].legend()

plt.tight_layout()
plt.show()

In [0]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

print("ANÁLISE DE CLUSTERS NATURAIS:")
X_top = df[top_pca_features + ['Amount']]
y = df['Class']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_top)

pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

print(f"Variância explicada pelo PCA 2D: {pca_2d.explained_variance_ratio_.sum():.3f}")

plt.figure(figsize=(12, 8))
scatter = plt.scatter(X_pca_2d[:, 0], X_pca_2d[:, 1], c=y, cmap='bwr', alpha=0.6, s=10)
plt.colorbar(scatter, label='Classe (0=Normal, 1=Fraude)')
plt.xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]:.3f})')
plt.ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]:.3f})')
plt.title('Visualização PCA 2D - Top Features + Amount', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.show()

In [0]:
print("ANÁLISE DE PADRÕES TEMPORAIS AVANÇADOS:")

df['hour_of_day'] = (df['Time'] // 3600) % 24
df['day_cycle'] = np.sin(2 * np.pi * df['hour_of_day'] / 24)
df['day_cycle_cos'] = np.cos(2 * np.pi * df['hour_of_day'] / 24)

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

scatter1 = axes[0,0].scatter(df['hour_of_day'], df['Amount'], c=df['Class'], 
                            cmap='bwr', alpha=0.5, s=10)
axes[0,0].set_xlabel('Hora do Dia')
axes[0,0].set_ylabel('Amount')
axes[0,0].set_title('Amount vs Hora do Dia', fontweight='bold')
plt.colorbar(scatter1, ax=axes[0,0], label='Classe')

hourly_amount = df.groupby('hour_of_day').agg({'Amount': 'mean', 'Class': 'mean'})
axes[0,1].plot(hourly_amount.index, hourly_amount['Amount'], marker='o', linewidth=2)
axes[0,1].set_xlabel('Hora do Dia')
axes[0,1].set_ylabel('Amount Médio', color='blue')
axes[0,1].tick_params(axis='y', labelcolor='blue')
axes[0,1].set_title('Amount Médio e Taxa de Fraude por Hora', fontweight='bold')
axes[0,1].grid(True, alpha=0.3)

ax2 = axes[0,1].twinx()
ax2.plot(hourly_amount.index, hourly_amount['Class'] * 100, marker='s', 
         color='red', linewidth=2)
ax2.set_ylabel('Taxa de Fraude (%)', color='red')
ax2.tick_params(axis='y', labelcolor='red')

fraud_hour_amount = df[df['Class'] == 1][['hour_of_day', 'Amount']]
axes[1,0].hexbin(fraud_hour_amount['hour_of_day'], fraud_hour_amount['Amount'], 
                gridsize=20, cmap='Reds', alpha=0.8)
axes[1,0].set_xlabel('Hora do Dia')
axes[1,0].set_ylabel('Amount')
axes[1,0].set_title('Densidade de Fraudes: Hora vs Amount', fontweight='bold')

fraud_cyclic = df[df['Class'] == 1]['day_cycle']
normal_cyclic = df[df['Class'] == 0]['day_cycle']
axes[1,1].hist(fraud_cyclic, bins=24, alpha=0.7, color='red', density=True, label='Fraude')
axes[1,1].hist(normal_cyclic, bins=24, alpha=0.3, color='blue', density=True, label='Normal')
axes[1,1].set_xlabel('Ciclo Circadiano (seno)')
axes[1,1].set_ylabel('Densidade')
axes[1,1].set_title('Distribuição do Ciclo Circadiano', fontweight='bold')
axes[1,1].legend()

plt.tight_layout()
plt.show()